# Replacement Score Test

## Load the model from the local checkpoint (save from HF before)

In [1]:
import pathlib
import shutil
from circuit_tracer.transcoder.cross_layer_transcoder import load_clt

from crosslayer_transcoder.utils.model_converters.circuit_tracer import (
    CircuitTracerConverter,
)
from transformer_lens import HookedTransformerConfig
from transformer_lens.loading_from_pretrained import get_pretrained_model_config

In [ ]:
import os
os.environ["HF_TOKEN"] = "ADD_TOKEN_HERE"

In [3]:


from crosslayer_transcoder.utils.module_builder import build_module_from_config, yaml_to_config
config = yaml_to_config("../../config/topk-clt-debug.yaml")
clt_module = build_module_from_config(config)

save_dir = pathlib.Path("clt_module_test")
feature_input_hook = "hook_resid_pre"
feature_output_hook = "hook_mlp_out"

converter = CircuitTracerConverter(
save_dir=save_dir,
feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
)
converter.convert_and_save(clt_module)

model {'class_path': 'crosslayer_transcoder.model.clt.CrossLayerTranscoder', 'init_args': {'encoder': {'class_path': 'crosslayer_transcoder.model.clt.Encoder', 'init_args': {'d_acts': 768, 'd_features': 10000, 'n_layers': 12}}, 'decoder': {'class_path': 'crosslayer_transcoder.model.clt.CrosslayerDecoder', 'init_args': {'d_acts': 768, 'd_features': 10000, 'n_layers': 12}}, 'nonlinearity': {'class_path': 'crosslayer_transcoder.model.topk.PerLayerTopK', 'init_args': {'k': 16}}, 'input_standardizer': {'class_path': 'crosslayer_transcoder.model.standardize.DimensionwiseInputStandardizer', 'init_args': {'n_layers': 12, 'activation_dim': 768}}, 'output_standardizer': {'class_path': 'crosslayer_transcoder.model.standardize.DimensionwiseOutputStandardizer', 'init_args': {'n_layers': 12, 'activation_dim': 768}}}}
encoder {'class_path': 'crosslayer_transcoder.model.clt.Encoder', 'init_args': {'d_acts': 768, 'd_features': 10000, 'n_layers': 12}}
d_acts 768
d_features 10000
n_layers 12
decoder {'cl

Loading dataset shards:   0%|          | 0/80 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1217 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1174 > 1024). Running this sequence through the model will result in indexing errors


dead_features {'class_path': 'crosslayer_transcoder.metrics.dead_features.DeadFeatures', 'init_args': {'n_features': 10000, 'n_layers': 12, 'return_per_layer': True, 'return_log_freqs': True, 'return_neuron_indices': True}}
n_features 10000
n_layers 12
return_per_layer True
return_log_freqs True
return_neuron_indices True
learning_rate 3e-4
compile True
lr_decay_step 16000
lr_decay_factor 0.1
topk_aux {'class_path': 'crosslayer_transcoder.model.topk.PerLayerTopK', 'init_args': {'k': 512}}
k 512
tokens_till_dead 500000
aux_loss_scale 0.03125
compute_dead_features True
compute_dead_features_every 500


Token indices sequence length is longer than the specified maximum sequence length for this model (1561 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2027 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2459 > 1024). Running this sequence through the model will result in indexing errors
/root/crosslayer-transcoder/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'topk_aux' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['topk_aux'])`.


In [4]:
transcoder = load_clt(
    clt_path=save_dir.as_posix(),
    lazy_decoder=False,
    lazy_encoder=False,
    feature_input_hook=feature_input_hook,
    feature_output_hook=feature_output_hook,
)


In [5]:
import torch
config = get_pretrained_model_config("gpt2")
config.d_type = torch.float16

In [6]:
from circuit_tracer import ReplacementModel
rm = ReplacementModel.from_config(
    config=config,
    transcoders=transcoder,
)



In [ ]:
prompt = (
    "The capital of state containing Dallas is"  # What you want to get the graph for
)
max_n_logits = 10  # How many logits to attribute from, max. We attribute to min(max_n_logits, n_logits_to_reach_desired_log_prob); see below for the latter
desired_logit_prob = 0.95  # Attribution will attribute from the minimum number of logits needed to reach this probability mass (or max_n_logits, whichever is lower)
max_feature_nodes = 8192  # Only attribute from this number of feature nodes, max. Lower is faster, but you will lose more of the graph. None means no limit.
batch_size = 256  # Batch size when attributing
offload = (
    "cpu"
)  # Offload various parts of the model during attribution to save memory. Can be 'disk', 'cpu', or None (keep on GPU)
verbose = True  # Whether to display a tqdm progress bar and timing report


In [8]:
from pathlib import Path
import torch

from circuit_tracer import ReplacementModel, attribute
from circuit_tracer.utils import create_graph_files

torch.cuda.empty_cache()

# FOr some reason this takes up a lot of VRAM
graph = attribute(
    prompt=prompt,
    model=rm,
    max_n_logits=max_n_logits,
    desired_logit_prob=desired_logit_prob,
    batch_size=batch_size,
    max_feature_nodes=max_feature_nodes,
    offload=offload,
    verbose=verbose
)

Phase 0: Precomputing activations and vectors


OutOfMemoryError: CUDA out of memory. Tried to allocate 15.62 GiB. GPU 0 has a total capacity of 31.37 GiB of which 7.31 GiB is free. Including non-PyTorch memory, this process has 24.05 GiB memory in use. Of the allocated memory 21.92 GiB is allocated by PyTorch, and 1.54 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
graph_dir = 'graphs'
graph_name = 'example_graph.pt'
graph_dir = Path(graph_dir)
graph_dir.mkdir(exist_ok=True)
graph_path = graph_dir / graph_name

graph.to_pt(graph_path)